In [24]:
library(dplyr) # to use the sample_n function

### STAT854: Final Project Code
#### Senad Kokic & Zack Huang

In this notebook, we aim to implement the pseudo-population bootstrap proposed by Gross (1980) and Chen et al. (2019). 

In the naive bootstrap, the first step is as follows (noting a stratified sampling design):

- Let $b = 1$. Independently in each of the $H$ strata, draw a simple random sample _with_ replacement of size $n_h$ from the original sample $\{y_{h1}, \cdots, y_{h_{n_h}}\}$ to obtain the bootstrap sample $\{y_{h1}^{*b}, \cdots, y_{h_{n_h}}^{*b}\}$ with corresponding weights $\{w_{h1}^{*b}, \cdots, w_{h_{n_h}}^{*b}\}$

This leads to the biased variance estimate, $\widehat{V}^*(\bar{y}_{Ha}) = \frac{1}{B - 1}\sum_{b = 1}^{B}(\bar{y}_{Ha}^{*b} - \bar{y}_{Ha})$.

So, the method we employ utilizes sampling _without_ replacement. 

We also note the assignment information:

**Your R code must be a function, where the key arguments are the weights, strata information, and $B$ (the number of bootstrap replicates) and the key output is a matrix/data frame containing the bootstrap weights**

Here:
- Rescale the weights (if we do not rescale the weights, we will have too many replicates)
- Give each column a unique ID for identification (to combine into bootstrap weights later)

We also refer to the Assignment 2 file to determine the $N_h$ values for each stratum $h$.

In [18]:
# read in the data
syc = read.table("/Users/senadkokic/Desktop/W2024/STAT 854/Final Project/Data/syc.txt",
            header = T, sep = ",")
# rescale the final weight
syc$finalwt = syc$finalwt*(23655/25012)
# add a unique ID column for each row
ids = seq(from = 1, to = nrow(syc), by = 1)
# combine the ID column and the remaining data
syc = cbind(ids, syc)
# save the Nhs as a vector (from assignment 2 file) for step 2
Nhs = c(2724, 3192, 4107, 2705, 3504, 376, 56, 528, 624, 520, 672, 384, 744, 847, 824, 1848)
# save the nhs as a vector for step 3
nhs = c()
for (i in 1:length(unique(syc$stratum))) {
    nhs = append(nhs, nrow(syc[syc$stratum == i, ]))
}

First, we must consider the first step of the process - constructing the fixed part of the pseudo-population, $U^f$. 

This entails repeating the pair $(y_i, \pi_i)$ a total of $\left \lfloor{\pi_i^{-1}}\right \rfloor$ (i.e. $\left \lfloor{w_i}\right \rfloor$ times), for all $i \in S$. For our purposes, this also requires repetition of the ID. 

However, we ran into mathematical difficulty using the final weights (`finalwt`) as given. So, seeing as we have the appropriate information for $N_h$ and $n_h$, we instead repeat the pair $(y_i, \pi_i)$ a total of $\left \lfloor{\frac{N_h}{n_h}}\right \rfloor$ times.

We define a helper function, `construct_fixed_pseudo` to do this.

In [25]:
construct_fixed_pseudo = function(ids, stratum, weights, Nh, nh) {
    # create a blank list for weights, stratum info, and ids
    Weights = c()
    Stratum = c()
    ID = c()
    for (i in 1:length(weights)) {
        # step 1 - take the floor
        num_reps = floor(Nhs[stratum[i]]/nhs[stratum[i]])
        # step 2 - replicate each weight, stratum, id num_reps times
        reps = rep(weights[i], times = num_reps)
        reps_strat = rep(stratum[i], times = num_reps)
        reps_id = rep(ids[i], times = num_reps)
        # step 3 - add these to the replicates lists
        Weights = append(Weights, reps)
        Stratum = append(Stratum, reps_strat)
        ID = append(ID, reps_id)
    }
    # return a dataframe of the above
    return(data.frame(cbind(ID, Stratum, Weights)))
}

Simply checking that the function works as desired. 

In [41]:
fixed_pseudo = construct_fixed_pseudo(syc$ids, syc$stratum, syc$finalwt)

Now, we move to the second part of the process - completing the pseudo-population, $U^*$. 

In this step, we draw $U^{c*}$ from $\{(y_i, \pi_i)\}_{i \in S}$ using the original sampling design that led to $S$, leading to $U^* = U^f \cup U^{c*}$. 

We note that the original survey we are considering, the Survey of Youth in Custody Survey, used a stratified sampling design. In this survey, 16 strata were considered, and the information is given. 

To determine how many we should be sampling, we consider the difference across each $N_h$ and $\widehat{N}_h$, where $\widehat{N}_h$ is the current stratum size in the pseudo-population. We ideally want to sample $N_h -  \widehat{N}_h < n_h$ from the original sample. 

Assuming this is stratified simple random sampling, we construct this sample without replacement based on the strata information. To this end, we use a helper function, `construct_stsrswor_pseudo`.

Here, we use the `slice_sample` function from the `dplyr` library. It has the same functionality as the `sample` function, however directly returns the samples from the dataframe instead of indices for sampling. 

In [42]:
construct_stsrswor_pseudo = function(sample_data, fixed_pseudo_population, stratum_sizes) {
    # set data to contain U^c*
    Uc = c()
    # want to iterate through each stratum
    for (strata in 1:length(unique(sample_data$stratum))) {
        # retrieve Nh from the stratum sizes
        Nh = stratum_sizes[strata]
        # determine Nh_hat (the present number of observations in stratum h)
        Nh_hat = nrow(fixed_pseudo_population[fixed_pseudo_population$Stratum == strata, ])
        # determine how many remain to be sampled 
        n_r = Nh - Nh_hat
        # skip/do not consider if n_r is less than or equal to 0
        if (n_r <= 0) next
        # now, extract the data from the sample that corresponds to stratum h
        strata_data = sample_data[sample_data$stratum == strata, ]
        # extract the inclusion probabilities
        weights = 1/strata_data$finalwt
        # sample from the associated sample stratum
        sample = slice_sample(.data = strata_data, n = n_r, replace = FALSE, weight_by = weights)
        # combine with the previous samples
        Uc = rbind(Uc, sample)
    }
    # return Uc as the IDs, Stratum, Weights
    ID = Uc$ids
    Stratum = Uc$stratum
    Weights = Uc$finalwt
    # make a dataframe
    Uc = data.frame(cbind(ID, Stratum, Weights))
    return(Uc)
}

Simply checking that the function works as desired. 

In [43]:
var_pseudo = construct_stsrswor_pseudo(syc, fixed_pseudo, Nhs)

Now, we construct another helper, `combine_pseudo`, to combine the results of the previous two functions.

In [44]:
combine_pseudo = function(fixed_pp, var_pp) {
    # combine the pseudo-population components
    return(rbind(fixed_pp, var_pp))
}

Simply checking that the function works as desired. 

In [46]:
pseudo_pop = combine_pseudo(fixed_pseudo, var_pseudo)

Finally, we consider the third step - being selecting a sample $S^*$ from $U^*$ in the same manner that $S$ was selected from $U$. In our case, this would be using STSRSWOR, sampling $n_h$ units for each stratum $h$. 

We construct a function, `bootstrap_sample`, to do this.

In [48]:
bootstrap_sample = function(pseudo_population, nh = nhs) {
    # create a list for the combined sample
    sample_hs = c()
    for (i in 1:length(nh)) {
        # selecting the ith strata from the pseudo population for consideration
        x = pseudo_population[pseudo_population$Stratum == i, ]
        # determine the weights
        wts = 1/x$Weights
        # select the sample
        sample = slice_sample(.data = x, 
                               n = nhs[i], 
                               replace = FALSE, 
                               weight_by = wts)
        # attach to the existing overall sample
        sample_hs = rbind(sample_hs, sample)
    }
    return(sample_hs)
}

Simply checking that the function works as desired. 

In [ ]:
bootstrap_sample(pseudo_pop)

Now, we put it all together - combining the helper functions into one function to create $B$ bootstrap samples. So, the desired output here is a weight matrix, being 2621x100. 

In constructing the weight matrix, we recognize the main jist of bootstrapping is the sampling with replacement. We must figure how a boostrap weight, $w_{h_{n_i}}^{*b}$ is constructed.

If a sample appears multiple times in the bootstrap, say $t$ times, its weight is $w_{h_{n_i}}^{*b} = tw_{h_{n_i}}$, otherwise, it has a weight of $0$.

In [97]:
chen_bootstrap = function(ids = syc$ids , # the ID values extracted from the survey file
                          stratum = syc$stratum, # the stratum values from the survey file
                          weights = syc$finalwt, # weights from the survey file
                          Nh = Nhs, # stratum sizes
                          nh = nhs, # stratum sample sizes
                          sample_data = syc, # sample data
                          B = 100) # number of bootstrap replicates
{
    # create the row names
    col_nam = c("ID")
    # save a list for the weight matrix: the first column will be the id column
    weight_matrix = matrix(nrow = nrow(sample_data), ncol = B)
    # creating the fixed part of the pseudo population
    fixed_pseudo = construct_fixed_pseudo(ids, stratum, weights)
    # create the variable part of the pseudo population
    var_pseudo = construct_stsrswor_pseudo(sample_data, fixed_pseudo, Nh)
    # combine the two parts of the pseudo population
    pseudo_pop = combine_pseudo(fixed_pseudo, var_pseudo)
    # now, get the bootstrap weights
    for (i in 1:B) {
        # create a single bootstrap sample
        boot = bootstrap_sample(pseudo_pop)
        # now, compute the weights based on the ids
        for (id in 1:nrow(sample_data)) {
            # if in sample, sum of weights. otherwise, 0
            if (id %in% boot$ID) {
                weight_matrix[id, i] = sum(boot$Weights[boot$ID == id])
            } else {
                weight_matrix[id, i] = 0
            }
        }
        # add the corresponding weight vector name to column names
        col_nam = append(col_nam, paste("w", i, sep = ""))
    }
    # combine the ids and the weights into a single weight matrix
    weight_matrix = cbind(ids, weight_matrix)
    # make it a dataframe
    weight_df = data.frame(weight_matrix)
    # set the column names
    colnames(weight_df) = col_nam
    return(weight_df)
}

Now, we finally produce the weight matrix.

In [103]:
# setting a seed
set.seed(454854)
# call the function 
weight_df = chen_bootstrap(ids = syc$ids , # the ID values extracted from the survey file
                          stratum = syc$stratum, # the stratum values from the survey file
                          weights = syc$finalwt, # weights from the survey file
                          Nh = Nhs, # stratum sizes
                          nh = nhs, # stratum sample sizes
                          sample_data = syc, # sample data
                          B = 100) # number of bootstrap replicates
# write the dataframe to a csv
write.csv(weight_df, 
          file = "/Users/senadkokic/Desktop/W2024/STAT 854/Final Project/Data/weight_matrix.csv",
          row.names = FALSE)